In [1]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba

/Users/michaelweinold/github/brightwebapp/.venvbwa/lib/python3.13/site-packages/bw2calc/__init__.py:50: UserWarning: 
It seems like you have an ARM architecture, but haven't installed scikit-umfpack:

    https://pypi.org/project/scikit-umfpack/

Installing it could give you much faster calculations.

  warnings.warn(UMFPACK_WARNING)


In [27]:
bd.projects.set_current("test")

In [26]:
bd.projects.delete_project(name='test', delete_dir=True or False)

'USEEIO-1.1'

In [28]:
bd.projects

Brightway2 projects manager with 6 objects:
	USEEIO-1.1
	dev
	ei_3_10
	fixture
	supply chain graph
	test
Use `projects.report()` to get a report on all projects.

In [29]:
bd.databases

Databases dictionary with 0 objects

In [30]:
del bd.databases["steel bike"]

KeyError: 

In [31]:
sb = bd.Database("steel bike")
sb.register()

In [32]:
coal = sb.new_node(name="Coal", unit="kg", code="coal")
steel = sb.new_node(name="Steel", unit="kg", code="steel")
iron_mining = sb.new_node(name="Iron mining", unit="kg", code="im")
iron_ore = sb.new_node(name="Iron ore", unit="kg", code="io", type="emission")
steel_bike = sb.new_node(name="Steel bike", unit="unit", code="sb")

In [33]:
coal.save()
steel.save()
iron_mining.save()
iron_ore.save()
steel_bike.save()

In [34]:
iron_mining.new_edge(
    input=iron_ore,
    amount=1.25, 
    uncertainty_type=5, 
    minimum=1,
    maximum=1.75, 
    type='biosphere',
).save()

In [35]:
steel.new_edge(
    input=iron_mining,
    amount=1.1, 
    uncertainty_type=5, 
    minimum=1,
    maximum=1.25, 
    type='technosphere',
).save()

In [36]:
steel.new_edge(
    input=coal,
    amount=0.2, 
    uncertainty_type=5, 
    minimum=0.1,
    maximum=0.5, 
    type='technosphere',
).save()

In [37]:
steel_bike.new_edge(
    input=steel,
    amount=8, 
    uncertainty_type=5, 
    minimum=4,
    maximum=15, 
    type='technosphere',
).save()

In [43]:
iron_mining.new_edge(
    input=steel_bike,
    amount=0.001, 
    type='technosphere',
).save()

In [50]:
[e for e in bd.get_node(code='sb').edges()]

[Exchange: 8 kg 'Steel' (kg, GLO, None) to 'Steel bike' (unit, GLO, None)>]

coal.new_edge(
    input=steel_bike,
    amount=0.001, 
    type='technosphere',
).save()
steel.new_edge(
    input=steel_bike,
    amount=0.005, 
    type='technosphere',
).save()

In [42]:
[i.as_dict() for i in bd.Database("steel bike")]

[{'database': 'steel bike',
  'code': 'sb',
  'location': 'GLO',
  'name': 'Steel bike',
  'unit': 'unit',
  'id': 169633624360030208},
 {'database': 'steel bike',
  'code': 'im',
  'location': 'GLO',
  'name': 'Iron mining',
  'unit': 'kg',
  'id': 169633624334864384},
 {'database': 'steel bike',
  'code': 'coal',
  'location': 'GLO',
  'name': 'Coal',
  'unit': 'kg',
  'id': 169633624297115648},
 {'database': 'steel bike',
  'code': 'steel',
  'location': 'GLO',
  'name': 'Steel',
  'unit': 'kg',
  'id': 169633624322281472},
 {'database': 'steel bike',
  'code': 'io',
  'location': 'GLO',
  'name': 'Iron ore',
  'unit': 'kg',
  'type': 'emission',
  'id': 169633624347447296}]

In [38]:
bd.Method(("resources",)).write([
    (iron_ore.key, {'amount': 3.141, 'uncertainty_type': 3, 'loc': 3.141, 'scale': 0.5926535}),
])

In [44]:
lca = bc.LCA({steel_bike: 1}, ("resources",))
lca.lci()
lca.lcia()

NonsquareTechnosphere: Technosphere matrix is not square: 3 activities (columns) and 4 products (rows). Use LeastSquaresLCA to solve this system, or fix the input data

In [16]:
lca.biosphere_matrix.todense()

matrix([[0.  , 0.  , 1.25, 0.  ]])